<h2>FT005</h2>
<p> En este apartado se trata la base de datos FT005 </p>

In [ ]:
#Cargamos los paquetes que vamos a necesitar
import pandas as pd
import numpy as np
import pyreadstat
import pyarrow.feather as feather

In [ ]:
%%time
# Se carga el FT005 completo desde el archivo .dta
FT005_2020 = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/Informe_Pagos/FT005 20202021.dta')

In [ ]:
# Eliminamos los años que no nos interesa del FT007
# Se encuentran las observaciones que cumplen las condiciones
df_filtered = FT005_2020[
    (FT005_2020['Año'] == '2021')
    ].index

In [ ]:
# Se eliminan las observaciones que cumplen las condiciones anteriore
FT005_2020 = FT005_2020.drop(df_filtered)

In [ ]:
%%time
# Se carga el FT005 completo desde el archivo .dta
FT005_2021 = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/Informe_Pagos/FT005_202104.dta')

In [ ]:
FT005_2021['Nit'] = FT005_2021['Nit'].str.replace(r'ï»¿', '')

In [ ]:
# Se modifican los nombres de las columnas para posterimente pegar las bases de datos
FT005_2021.rename(columns={'razonsocial': 'RazonSocial'}, inplace=True)

In [ ]:
# Se eliminan las columnas que no son de interés
FT005_2021 = FT005_2021.drop(['Id','IdGnMaestroArchivo'], axis=1)

In [ ]:
# Se concatenan la bases de datos para encontrar la base de datos final
frames = [FT005_2021, FT005_2020]
FT005 = pd.concat(frames, ignore_index=True)

In [ ]:
FT005.columns

In [ ]:
del [FT005_2021, FT005_2020, df_filtered, frames]

In [ ]:
#Se transforman las variables a STR para mejorar su manipulación
# FT005.Nit = FT005.Nit.astype(str)

In [ ]:
#Se elije una sola EPS para hacer más fácil el análisis
# FT005_V1 = FEBRERO.loc[FEBRERO['Nit'] == '805000427']

In [ ]:
# Para guardar la base de datos ejemplo a tratar
# FT005_V1.to_excel(r'FT005_V1.xlsx', index = False)

In [ ]:
# Se elimina el FT005 full para liberar espacio en la memoria
# del FT005

In [ ]:
# Se carga la base de datos con una sola EPS para agilizar el código
# FT005 = pd.read_excel('FT005_V1.xlsx', dtype={'Nit':object, 'Año': object, 'idProveedor':object, 'Periodo':object})

In [ ]:
%%time
# Se reemplazan los nombres de las columnas para facilitar el tratamiento

FT005.rename(columns={'Nit': 'NIT_EPS', 'Año': 'Ano', 'Periodo': 'Mes', 'lineaNegocio': 'Linea','tipoIdProveedor': 'Tipoproveedor', 'idProveedor': 'Nitproveedor', 'codigoConcepto': 'Concepto', 'descripcionCyG': 'CyG', 'valorCausado': 'VCausado', 'valorPagado': 'VPagado'}, inplace=True)

In [ ]:
# Se cuenta el número de duplicados para hacer la respectiva corrección del tipo id deudor
df_razonsocial = FT005.groupby(['Nitproveedor', 'nombreProveedor']).size().reset_index().rename(columns={0:'Cuenta_FT05'})

In [ ]:
# Para guardar la base de datos lista para ser tratada en formato Excel y CSV
df_razonsocial.to_excel(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/df_razonsocial_FT005_V2.xlsx', index = False)

In [ ]:
#Se transforman las variables a STR para mejorar su manipulación
FT005[['NIT_EPS', 'Nitproveedor', 'Tipoproveedor', 'Mes', 'Ano']] = FT005[['NIT_EPS', 'Nitproveedor', 'Tipoproveedor', 'Mes', 'Ano']].astype(str)

In [ ]:
# Se filtra la línea de negocio de interés. En este caso corresponde a la línea 1
FT005 = FT005.loc[FT005['Linea'] == 1]

In [ ]:
# Para hacer la suma de Valor pagado y Valor causado por NIT EPS, tipo de proveedor, NIT proveedor y Mes de interés
FT005 = FT005.groupby(['NIT_EPS', 'Tipoproveedor', 'Nitproveedor', 'Mes', 'Ano']).agg({'VPagado': 'sum','VCausado': 'sum'}).reset_index()

In [ ]:
# Se comprueba que no haya duplicados cuando se filtra por NIT EPS, tipo de proveedor, NIT proveedor y Mes. Para esto se cuenta el número de duplicados
duplicadosFT005 = FT005[FT005.duplicated(['NIT_EPS', 'Nitproveedor', 'Mes', 'Ano'], keep='last')]
print("El número de registros duplicados es:", duplicadosFT005.NIT_EPS.count(), sep='\n')

In [ ]:
# Se genera una nueva columna extrayendo el primer caracter del ID nitproveedor
FT005['d1'] = FT005['Nitproveedor'].str[:1]

In [ ]:
# Se genera una nueva columna con la cuenta del numero de componentes que tiene el ID nitproveedor
FT005['largo'] = FT005.Nitproveedor.str.len()

In [ ]:
# Se cuenta el número de duplicados para hacer la respectiva corrección del tipo id deudor
df_tipoproveedor = FT005.groupby(['NIT_EPS', 'Nitproveedor', 'Mes', 'Ano']).size().reset_index().rename(columns={0:'d'})
# df_tipoiddeudor.sort_values(by='d', ascending=False)

In [ ]:
# Se crea la unión de del FT003 con la cuenta de repetidos según NIT EPS, NIT proveedor, mes y año
FT005 = FT005.merge(df_tipoproveedor, left_on=['NIT_EPS', 'Nitproveedor', 'Mes', 'Ano'], right_on=['NIT_EPS', 'Nitproveedor', 'Mes', 'Ano'], how='left')

In [ ]:
# Se limpia la clasificación por Tipo de Proveedor
FT005['Tipoproveedor_corregido'] = FT005['Tipoproveedor'] # Se crea la columna con categoria NA como base
# Condiciones
FT005.loc[(FT005['Tipoproveedor_corregido'] == 'NA') &
       (FT005['largo'] == 10) & # Para signar el tipo de proveedor CC
       (FT005['d1'] == '1'), 
       'Tipoproveedor_corregido'] = 'CC' 

FT005.loc[(FT005['Tipoproveedor_corregido'] == 'NA') &
       (FT005['largo'] == 9) & # Para signar el tipo de proveedor NI
       ((FT005['d1'] == '7') |
       (FT005['d1'] == '8') |
       (FT005['d1'] == '9')),  
       'Tipoproveedor_corregido'] = 'NI' 

FT005.loc[(FT005['Tipoproveedor_corregido'] == 'NA') &
       (FT005['Tipoproveedor_corregido'] != 'NI'), # Para completar el tipo de proveedor CC
       'Tipoproveedor_corregido'] = 'CC' 

FT005.loc[(FT005['Tipoproveedor_corregido'] == 'NA') &
       (FT005['largo'] == 9), # Para completar el tipo de proveedor CC
       'Tipoproveedor_corregido'] = 'CC' 

In [ ]:
# Se filtra el dataframe con las observaciones que cumplen las condiciones
FT004_V2 = FT005[
    (FT005['Tipoproveedor_corregido'] == 'NA')
    ]

In [ ]:
print(FT005.Tipoproveedor.value_counts())
print(FT005.Tipoproveedor_corregido.value_counts())

In [ ]:
print(FT005.Tipoproveedor.value_counts())
print(FT005.Tipoproveedor_corregido.value_counts())

In [ ]:
# # Se crea una lista de condiciones simultaneas para asignar el tipo de proveedor

# condiciones = [
#     (FT005_V2['largo'] == 10) & (FT005_V2['d1'] == '1'), # Para signar el tipo de proveedor CC
#     (FT005_V2['largo'] == 9) & ((FT005_V2['d1'] == '7') | (FT005_V2['d1'] == '8') | (FT005_V2['d1'] == '9')), # Para signar el tipo de proveedor NI
#     (FT005_V2['largo'] == 9)
#     ]

# # Se asignan los valores correspondientes a cada una de las condiciones de nuestra lista
# valores = ['CC', 'NI', 'CC']

# # Se crea una nueva columna para asignar los valores dadas las condiciones 
# FT005_V2['tipoproveedor_corregido'] = np.select(condiciones, valores)
# FT005_V2.tipoproveedor_corregido = FT005_V2.tipoproveedor_corregido.astype(str)

# # Se da un resumen de como se ve el en general la nueva variable corregida
# print(FT005_V2.tipoproveedor.value_counts())
# print(FT005_V2.tipoproveedor_corregido.value_counts())

In [ ]:
# ESTA LINEA GENERA PROBLEMAS PORQUE CONVIERTE TODO EN CC Y UN NIT DEBE TENER SIEMPRE 9 DIGITOS MÁS 1 DIGITO DE VERIFICACION - PENSAR EN COMPLEMENTO PARA MEJORAR
# FT005_V2.loc[(FT005_V2['tipoproveedor_corregido2'] != 'NI'), 'tipoproveedor_corregido2'] = 'CC' # Para completar el tipo de proveedor CC

In [ ]:
# Para hacer la suma de Valor pagado y Valor causado por NIT EPS, tipo de proveedor corregido, NIT proveedor y Mes de interés
FT005 = FT005.groupby(['NIT_EPS', 'Tipoproveedor_corregido', 'Nitproveedor', 'Mes', 'Ano']).agg({'VPagado': 'sum','VCausado': 'sum'}).reset_index()

In [ ]:
# # Se carga la base de datos webgironoupc para pegarla con el FT005
# webgironoupc = pd.read_excel('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/GD_UPC_CC_PW_2020_ENEMAR2021_V2.xlsx', dtype={'NIT_EPS':object, 'Nitproveedor': object, 'Mes':object, 'Ano':object}) 
# # Se convierte la variable en mes y año
# webgironoupc[['Ano', 'Mes']] = webgironoupc[['Ano', 'Mes']].astype(float).astype(int).astype(str)
# # Se unen las dos bases de datos para obtener la información de webgironoupc (Giro Directo - No_UPC - Compra de Cartera) y FT005 en una sola base de datos
# FT005_web= pd.merge(FT005_V3, webgironoupc, how='outer', left_on=['NIT_EPS', 'Nitproveedor', 'Mes', 'Ano'], right_on=['NIT_EPS', 'Nitproveedor', 'Mes', 'Ano'])

In [ ]:
# Se eliminan la bases que no se necesitan para liberar espacio en la memoria# Se eliminan la bases que no se necesitan para liberar espacio en la memoria
del [duplicadosFT005]

In [ ]:
FT005.rename(columns={'Tipoproveedor_corregido': 'Tipoproveedor'}, inplace=True)

In [ ]:
# Se convierte la variable en mes y año
FT005[['Ano', 'Mes']] = FT005[['Ano', 'Mes']].astype(float).astype(int)
# Se agrega la información por NIT_EPS, Nitproveedor, noID, Departamento, Ano, Mes para las variables VPagado, VCausado, PW, Giro, No_UPC, Compra_Cartera
FT005 = FT005.groupby(['NIT_EPS', 'Tipoproveedor', 'Nitproveedor', 'Mes', 'Ano'])[['VPagado', 'VCausado']].sum().reset_index()

In [ ]:
%%time
# Se crea el rango de meses necesarios en cada mes
fecha_ideal=range(1, 12 + 1)
# Se rellena con ceros la información para los meses faltantes
FT005 = FT005.set_index('Mes').groupby(['NIT_EPS', 'Tipoproveedor', 'Nitproveedor', 'Ano']).apply(lambda x: x.reindex(index=fecha_ideal, fill_value=0)).drop(['NIT_EPS', 'Tipoproveedor', 'Nitproveedor', 'Ano'], 1)

In [ ]:
# FT005_EPS = FT005.groupby(['NIT_EPS', 'Mes', 'Ano']).agg({'VPagado':'sum', 'VCausado':'sum'}).reset_index()
# #HASTA AQUI VAMOS BIEN

In [ ]:
%%time
# Se calcula la diferencia mensual para conocer el valor mensual de cada cuenta ya que por defecto vienen agregadas
FT005_dif = FT005.groupby(['NIT_EPS', 'Nitproveedor', 'Tipoproveedor', 'Ano']).diff().fillna(0).reset_index()

In [ ]:
FT005 = FT005.reset_index()

In [ ]:
FT005_dif.loc[FT005_dif['Mes'] == 1] = FT005

In [ ]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT005_dif.rename(columns={'VPagado': 'pagomes', 'VCausado': 'causmes'}, inplace=True)

In [ ]:
FT005_EPS = FT005_dif.groupby(['NIT_EPS', 'Mes', 'Ano']).agg({'causmes':'sum', 'pagomes':'sum'}).reset_index()
#HASTA AQUI VAMOS BIEN

In [ ]:
# Se guarda un back up del FT005 antes de aplicar las diferencias
# Para guardar la base de datos lista para ser tratada en formato Excel y CSV
FT005_dif.to_csv(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/FT005_dif_backup_2.csv', index = False, encoding='utf-8-sig')
FT005.to_csv(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/FT005_backup_2.csv', index = False, encoding='utf-8-sig')
# feather
feather.write_feather(FT005_dif, 'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/FT005_dif_backup_feather_2')
feather.write_feather(FT005, 'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/FT005_backup_feather_2') # Esta línea guarda la base de datos en binario para optimizar el tiempo de cargay el espacio

In [ ]:
FT005 = FT005_dif

In [ ]:
del [FT005_dif, fecha_ideal, FT005_EPS]

In [ ]:
# Aquí se recupera el back up
FT005 = pd.read_feather('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/FT005_dif_backup_feather')

In [ ]:
# Se carga la base de datos prestadores.dta para su posterior pega
prestadores = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/prestadores.dta')

# Se carga la base de datos operadores.dta para su posterior pega
operadores = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/operadores.dta')

In [ ]:
# Se unen las dos bases de datos para obtener la información de FT005 con webgironoupc (Giro Directo - No_UPC - Compra de Cartera) y prestadores
FT005 = pd.merge(FT005, prestadores, how='left', left_on=['Nitproveedor'], right_on=['nitproveedor'])

In [ ]:
# Se unen las dos bases de datos para obtener la información de FT005 con webgironoupc (Giro Directo - No_UPC - Compra de Cartera) y prestadores
FT005 = pd.merge(FT005, operadores, how='left', left_on=['Nitproveedor'], right_on=['nitproveedor'])

In [ ]:
# Se eliminan la bases que no se necesitan para liberar espacio en la memoria
del [operadores, prestadores]

In [ ]:
# Se crea la columna noID tomando como referencia la columna clpr_nombre
FT005['noID'] = FT005['clpr_nombre']

In [ ]:
# Para conocer las diferencias entre las correciones
print(FT005.noID.value_counts(dropna=True))

In [ ]:
# Se establecen las condiciones para establecer correctamente el noID

FT005.loc[(FT005['NIT_EPS'] == FT005['Nitproveedor']), # Se reemplaza por EPS todo aquel proveedor que sea una EPS
       'noID'] = 'EPS'

FT005.loc[(FT005['Nitproveedor'] == '"901037916"') | # Se reemplazan los NIT de la nación
       (FT005['Nitproveedor'] == "900047282") |
       (FT005['Nitproveedor'] == "899999014") |
       (FT005['Nitproveedor'] == "900474727") |
       (FT005['Nitproveedor'] == "900462447"),  
       'noID'] = 'Nacion' 

In [ ]:
# Para conocer las diferencias entre las correciones
print(FT005.noID.value_counts(dropna=True))

In [ ]:
FT005.loc[FT005['noID'].isnull(), 'noID'] = 'Otro' # Para rellenar los vacios con "Otro"

In [ ]:
# Para conocer las diferencias entre las correciones
print(FT005.noID.value_counts(dropna=True))

In [ ]:
FT005.loc[(FT005['clpr_nombre'] == 'IPS') & # Para corregir el IPS cuando Tipo es vacío, por IPS-OL-DM-GIM
       (FT005['tipo'].notnull()), 
       'noID'] = 'IPS-OL-DM-GIM'

In [ ]:
# Para conocer las diferencias entre las correciones
print(FT005.noID.value_counts(dropna=True))

In [ ]:
FT005.loc[(FT005['tipo'].notnull()) & # Para dejar el Tipo cuando se tiene OSD u Otro
       ((FT005['noID'] == 'OSD') |
       (FT005['noID'] == 'Otro')), 
       'noID'] = FT005['tipo']

In [ ]:
# Para conocer las diferencias entre las correciones
print(FT005.noID.value_counts(dropna=True))

In [ ]:
FT005.loc[(FT005['noID'] == 'Otro') & # Para cambiar el noID por Persona Natural si en Tipo Proveedor se tiene CC y CE
       ((FT005['Tipoproveedor'] == 'CC') |
       (FT005['Tipoproveedor'] == 'CE')), 
       'noID'] = 'Persona Natural'

In [ ]:
# Para conocer las diferencias entre las correciones
print(FT005.noID.value_counts(dropna=True))

In [ ]:
FT005.loc[(FT005['departamento'].isnull()) & # Para asignar "Otro" al departamento si está vacío y si es una IPS o IPS Pública
       ((FT005['noID'] == 'IPS Pública') |
       (FT005['noID'] == 'IPS')), 
       'departamento'] = 'Otro'

In [ ]:
# Para conocer las diferencias entre las correciones
print(FT005.noID.value_counts(dropna=True))

In [ ]:
# FT005.loc[
#        (FT005['noID'] == 'Persona Natural'), 
#        'Nitproveedor'] = '123456789'

In [ ]:
# Aquí se agrega la base para poder pegar webgironoupc sin alterar la sumatoria por EPS. Después de esto se deben pegar las columnas departamento y noI

In [ ]:
FT005.columns

In [ ]:
# Para saber los duplicados que se tienen
duplicadosFT005 = FT005[FT005.duplicated(['NIT_EPS', 'Nitproveedor', 'Mes', 'Ano'], keep='last')]

In [ ]:
# Se filtra el dataframe con las observaciones que cumplen las condiciones
FT005_2 = FT005[
    (FT005['Nitproveedor'] == '176765')
    ]

In [ ]:
FT005_EPS = FT005.groupby(['NIT_EPS', 'Nitproveedor', 'noID', 'Mes', 'Ano']).agg({'pagomes':'sum', 'causmes':'sum'}).reset_index() 

In [ ]:
# Se guarda un back up del FT005 antes de aplicar las diferencias
# Para guardar la base de datos lista para ser tratada en formato Excel y CSV
FT005_EPS.to_csv(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/FT005_conNOID_backup.csv', index = False, encoding='utf-8-sig')
# feather
feather.write_feather(FT005_EPS, 'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/FT005_conNOID_backup_feather') # Esta línea guarda la base de datos en binario para optimizar el tiempo de cargay el espacio

In [ ]:
FT005_EPS = FT005.groupby(['NIT_EPS', 'Mes', 'Ano']).agg({'pagomes':'sum', 'causmes':'sum'}).reset_index() 
# HASTA AQUI VAMOS BIEN

In [ ]:
# Se agrupa la información de interés
FT005_V2 = FT005.groupby(['NIT_EPS', 'Nitproveedor', 'Ano', 'Mes']).agg({'pagomes':'sum', 'causmes':'sum'}).reset_index()

In [ ]:
# Se carga la base de datos webgironoupc para pegarla con el FT005
webgironoupc = pd.read_excel('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/GD_UPC_CC_PW_2020_ENEABR2021_V2.xlsx', dtype={'NIT_EPS':object, 'Nitproveedor': object, 'Mes':object, 'Ano':object}) 

In [ ]:
# Se convierte la variable en mes y año
webgironoupc[['Ano', 'Mes']] = webgironoupc[['Ano', 'Mes']].astype(float).astype(int).astype(str)
FT005_V2[['Ano', 'Mes']] = FT005_V2[['Ano', 'Mes']].astype(float).astype(int).astype(str)

In [ ]:
webgironoupc[['Nitproveedor']] = webgironoupc[['Nitproveedor']].astype(str)
FT005_V2[['Nitproveedor']] = FT005_V2[['Nitproveedor']].astype(str)

In [ ]:
webgironoupc = webgironoupc.groupby(['NIT_EPS', 'Nitproveedor', 'Ano', 'Mes']).agg({'Giro':'sum', 'No_UPC':'sum', 'Compra_Cartera':'sum', 'PW':'sum'}).reset_index()

In [ ]:
webgironoupc[['Ano', 'Mes']] = webgironoupc[['Ano', 'Mes']].astype(float).astype(int).astype(str)
webgironoupc[['Nitproveedor']] = webgironoupc[['Nitproveedor']].astype(str)

In [ ]:
# Se unen las dos bases de datos para obtener la información de webgironoupc (Giro Directo - No_UPC - Compra de Cartera) y FT005 en una sola base de datos
FT005_V2 = pd.merge(FT005_V2, webgironoupc, how='outer', left_on=['NIT_EPS', 'Nitproveedor', 'Mes', 'Ano'], right_on=['NIT_EPS', 'Nitproveedor', 'Mes', 'Ano'])

In [ ]:
FT005_EPS = FT005_V2.groupby(['NIT_EPS', 'Mes', 'Ano']).agg({'pagomes':'sum', 'causmes':'sum', 'Giro':'sum', 'No_UPC':'sum', 'Compra_Cartera':'sum', 'PW':'sum'}).reset_index() 
# HASTA AQUI VAMOS BIEN

In [ ]:
del [webgironoupc, FT005_EPS]

In [ ]:
# Se eliminan las columnas que no se requieren en el merge
FT005 = FT005.drop(['Tipoproveedor', 'nombre_prestador', 'nitproveedor_x', 'clpr_codigo',
       'clpr_nombre', 'naju_nombre', 'pagomes', 'causmes', 'nitproveedor_y', 'razonsocial', 'tipo'], axis=1)

In [ ]:
FT005[['NIT_EPS', 'Nitproveedor', 'Ano', 'Mes']] = FT005[['NIT_EPS', 'Nitproveedor', 'Ano', 'Mes']].astype(str)
FT005_V2[['NIT_EPS', 'Nitproveedor', 'Ano', 'Mes']] = FT005_V2[['NIT_EPS', 'Nitproveedor', 'Ano', 'Mes']].astype(str)

In [ ]:
# Se procede a identificar aquellos Nitproveedor con noID priorizados según el tratamiento inicial del FT005
duplicadosFT005 = FT005.drop_duplicates(['Nitproveedor', 'noID', 'departamento'], keep='first')

In [ ]:
# Se toman los valores únicos del Nitproveedor priorizando aquellos que tienen como noID EPS o Persona Natural
df_filtered = duplicadosFT005.sort_values('noID', ascending=False).drop_duplicates('Nitproveedor', keep='first')

In [ ]:
# Se eliminan las columnas que no se requieren en el merge
df_filtered = df_filtered.drop(['NIT_EPS', 'Ano', 'Mes'], axis=1)

In [ ]:
# Se comprueba que no haya duplicados cuando se filtra por NIT EPS, tipo de deudor, NIT proveedor, Mes y Año. Para esto se cuenta el número de duplicados
duplicadosFT005_V2 = duplicadosFT005[duplicadosFT005.duplicated(['Nitproveedor'], keep='last')]
print("El número de registros duplicados es:", duplicadosFT005_V2.Nitproveedor.count(), sep='\n')

In [ ]:
# Se unen las dos bases de datos para obtener las columnas de noID y Departamento
FT005_V2 = pd.merge(FT005_V2, df_filtered, how='left', left_on=['Nitproveedor'], right_on=['Nitproveedor'])

In [ ]:
# Guardar la base de datos de noID para posterior comparación
df_filtered.to_excel(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/noID_CausaPrestador_2020_2021_V2.xlsx', index = False)

In [ ]:
del [FT005, duplicadosFT005, duplicadosFT005_V2]

In [ ]:
# Para renombrar las columnas de interés
FT005_V2.rename(columns={'NIT_EPS': 'nit_eps', 'Nitproveedor': 'nitproveedor', 'Departamento': 'departamento', 'Ano': 'ano', 'Mes': 'mes', 'PW': 'pw', 'Giro': 'giro', 'No_UPC': 'no_upc', 'Compra_Cartera': 'compra_cartera'}, inplace=True)

In [ ]:
FT005 = FT005_V2.groupby(['nit_eps', 'mes', 'ano']).agg({'pagomes':'sum', 'causmes':'sum', 'giro':'sum', 'no_upc':'sum', 'compra_cartera':'sum', 'pw':'sum'}).reset_index()
# HASTA AQUÍ VAMOS BIEN

In [ ]:
# Hasta aquí no se ha podido hacer una correcta agregación del noID y departamento ya que cuando se agrega la información, las cuentas no cuadran

In [ ]:
FT005_V2.columns

In [ ]:
FT005_V3 = FT005_V2.groupby(['nit_eps', 'nitproveedor', 'ano', 'mes',]).agg({'pagomes':'sum', 'causmes':'sum', 'giro':'sum', 'no_upc':'sum', 'compra_cartera':'sum', 'pw':'sum'}).reset_index()

In [ ]:
FT005 = FT005_V3.groupby(['nit_eps', 'mes', 'ano']).agg({'pagomes':'sum', 'causmes':'sum', 'giro':'sum', 'no_upc':'sum', 'compra_cartera':'sum', 'pw':'sum'}).reset_index()
# HASTA AQUI VAMOS BIEN

In [ ]:
FT005_V3[['nit_eps', 'mes', 'ano']] = FT005_V3[['nit_eps', 'mes', 'ano']].astype(str)

In [ ]:
# Guardar la base de datos lista para ser tratada en formato Excel y CSV
# FT005.to_excel(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/CausaPrestador_2020_2021.xlsx', index = False)
FT005_V3.to_csv(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/CausaPrestador_2020_2021_V2.csv', index = False, encoding='utf-8-sig')

In [ ]:
# Se eliminan la bases que no se necesitan para liberar espacio en la memoria
del [BACKUP, df_filtered, FT005_V2, FT005_V3]